In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
complete_train = pd.read_csv("/kaggle/input/follow-the-trend-time-series-forecasting/train.csv")
df_test = pd.read_csv("/kaggle/input/follow-the-trend-time-series-forecasting/test.csv")
df_sample = pd.read_csv("/kaggle/input/follow-the-trend-time-series-forecasting/sample_submission.csv")

In [ ]:
df_train = complete_train.copy()
df_train.drop(["Country", "Product"], axis = 1, inplace = True)

In [ ]:
df_train = df_train.groupby("Sales_Date").mean()  # Use mean, sum, or another aggregation function
complete_train.head()

In [ ]:
# df_train["Sales_Date"] = pd.to_datetime(df_train["Sales_Date"], format="%Y-%m-%d")
# df_train = df_train.set_index("Sales_Date")

In [ ]:
df_train.index = pd.to_datetime(df_train.index)
df_train = df_train.asfreq('D')  # Daily


In [ ]:
df_train = df_train.ffill()
df_train.head()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
sd = seasonal_decompose(df_train)

sd.plot()
plt.show()


In [ ]:
sd.seasonal[:14].plot()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)

In [ ]:
target = "Sales_Qty"
X = complete_train[complete_train["Country"] == "Bonaire, Saint Eustatius and Saba"].drop([target,"Product"], axis = 1)
y = complete_train[target]
print(X.shape)

In [ ]:
df_train.shape

In [ ]:
limit = 1000
X_train = df_train.index[:limit]
X_test = df_train.index[limit:]
y_train = df_train[:limit]
y_test = df_train[limit:]

In [ ]:
X_train = X.index.to_frame(index=False)  # Convert DatetimeIndex to DataFrame
type(X_train)

In [ ]:
X_train.head()

In [ ]:
rf.fit(X_train.reset_index(), np.array(y_train).ravel())
y_pred = rf.predict(X_test)